# time — Clock Time

**Purpose**:	Functions for manipulating clock time.

The time module provides access to several different types of clocks, each useful for different purposes. 

The standard system calls like `time()` report the system “wall clock” time. 

The `monotonic()` clock can be used to measure elapsed time in a long-running process because it is guaranteed never to move backwards, even if the system time is changed. 

For performance testing, `perf_counter()` provides access to the clock with the highest available resolution to make short time measurements more accurate. 

The CPU time is available through `clock()`, and `process_time()` returns the combined processor time and system time.

## Comparing Clocks

Use `get_clock_info()` to access basic information about the current implementation, including the clock’s resolution.

In [ ]:
import textwrap
import time

available_clocks = [
    ('monotonic', time.monotonic),
    ('perf_counter', time.perf_counter),
    ('process_time', time.process_time),
    ('time', time.time),
]

for clock_name, func in available_clocks:
    print(textwrap.dedent("""\
    {name}:
        adjustable    : {info.adjustable}
        implementation: {info.implementation}
        monotonic     : {info.monotonic}
        resolution    : {info.resolution}
        current       : {current}
    """).format(
        name=clock_name,
        info=time.get_clock_info(clock_name),
        current=func())
    )


## Wall Clock Time

One of the core functions of the time module is `time()`, which returns the number of seconds since the start of the “epoch” as a floating point value.

In [17]:
print('The time is:', time.time())

print('The time is      :', time.ctime())
later = time.time() + 15
print('15 secs from now :', time.ctime(later))

The time is: 1675456075.162237
The time is      : Fri Feb  3 22:27:55 2023
15 secs from now : Fri Feb  3 22:28:10 2023


## Monotonic Clocks

Because `time()` looks at the system clock, and the system clock can be changed by the user or system services for synchronizing clocks across multiple computers, calling `time()` repeatedly may produce values that go forwards and backwards. 

This can result in unexpected behavior when trying to measure durations or otherwise use those times for computation. Avoid those situations by using `monotonic()`, which always returns values that go forward.

In [18]:
start = time.monotonic()
time.sleep(0.1)
end = time.monotonic()
print(f'start : {start :>9.2f}')
print(f'end   : {end :>9.2f}')
print(f'span  : {end - start :>9.2f}')

start : 284614.39
end   : 284614.50
span  :      0.11


## Processor Clock Time

While `time()` returns a wall clock time, `process_time()` returns processor clock time. 

The values returned from `process_time()` reflect the actual time used by the program as it runs.

In [19]:
# Data to use to calculate md5 checksums
import hashlib

file_path = r"C:\Users\Vadim\Documents\GitHub\Python\_Lessons_\MEFATHIM\Sylllabus-AdvancedPython\week4\exercises\ex_class.ipynb"
data = open(file_path, 'rb').read()

for _ in range(5):
    h = hashlib.sha1()
    print(time.ctime(), f': {time.time() :0.3f} {time.process_time() :0.3f}')
    for _ in range(300_000):
        h.update(data)
    cksum = h.digest()

Fri Feb  3 22:27:55 2023 : 1675456075.521 18.969
Fri Feb  3 22:27:56 2023 : 1675456076.638 19.656
Fri Feb  3 22:27:57 2023 : 1675456077.640 20.281
Fri Feb  3 22:27:58 2023 : 1675456078.535 20.844
Fri Feb  3 22:27:59 2023 : 1675456079.630 21.312


## Performance Counter

It is important to have a high-resolution monotonic clock for measuring performance. 

Determining the best clock data source requires platform-specific knowledge, which Python provides in `perf_counter()`.

In [20]:
import hashlib
import time

# Data to use to calculate md5 checksums
file_path = r"C:\Users\Vadim\Documents\GitHub\Python\_Lessons_\MEFATHIM\Sylllabus-AdvancedPython\week4\exercises\ex_class.ipynb"
data = open(file_path, 'rb').read()

loop_start = time.perf_counter()

for _ in range(5):
    iter_start = time.perf_counter()
    h = hashlib.sha1()
    for _ in range(300_000):
        h.update(data)
    check_sum = h.digest()
    now = time.perf_counter()
    loop_elapsed = now - loop_start
    iter_elapsed = now - iter_start
    print(time.ctime(), f': {iter_elapsed :0.3f} {loop_elapsed :0.3f}')
        
# As with monotonic(), the epoch for perf_counter() is undefined, and the values are meant to be used for comparing and computing values, not as absolute times.

Fri Feb  3 22:28:01 2023 : 0.848 0.849
Fri Feb  3 22:28:02 2023 : 1.131 1.981
Fri Feb  3 22:28:03 2023 : 1.095 3.077
Fri Feb  3 22:28:04 2023 : 0.906 3.983
Fri Feb  3 22:28:05 2023 : 0.833 4.817


## Time Components

Storing times as elapsed seconds is useful in some situations, but there are times when a program needs to have access to the individual fields of a date (year, month, etc.). 

The time module defines struct_time for holding date and time values with components broken out so they are easy to access. 

There are several functions that work with struct_time values instead of floats.

In [21]:
def show_struct(s):
    print('  tm_year :', s.tm_year)
    print('  tm_mon  :', s.tm_mon)
    print('  tm_mday :', s.tm_mday)
    print('  tm_hour :', s.tm_hour)
    print('  tm_min  :', s.tm_min)
    print('  tm_sec  :', s.tm_sec)
    print('  tm_wday :', s.tm_wday)
    print('  tm_yday :', s.tm_yday)
    print('  tm_isdst:', s.tm_isdst)


print('gmtime:')
show_struct(time.gmtime())
print('\nlocaltime:')
show_struct(time.localtime())
print('\nmktime:', time.mktime(time.localtime()))

# The gmtime() function returns the current time in UTC. 
# localtime() returns the current time with the current time zone applied. 
# mktime() takes a struct_time and converts it to the floating point representation.

gmtime:
  tm_year : 2023
  tm_mon  : 2
  tm_mday : 3
  tm_hour : 20
  tm_min  : 28
  tm_sec  : 5
  tm_wday : 4
  tm_yday : 34
  tm_isdst: 0

localtime:
  tm_year : 2023
  tm_mon  : 2
  tm_mday : 3
  tm_hour : 22
  tm_min  : 28
  tm_sec  : 5
  tm_wday : 4
  tm_yday : 34
  tm_isdst: 0

mktime: 1675456085.0


## Parsing and Formatting Times

The two functions strptime() and strftime() convert between struct_time and string representations of time values. 

There is a long list of formatting instructions available to support input and output in different styles. 

The complete list is documented in the library documentation for the time module.

This example converts the current time from a string to a struct_time instance and back to a string.



In [22]:
def show_struct(s):
    print('  tm_year :', s.tm_year)
    print('  tm_mon  :', s.tm_mon)
    print('  tm_mday :', s.tm_mday)
    print('  tm_hour :', s.tm_hour)
    print('  tm_min  :', s.tm_min)
    print('  tm_sec  :', s.tm_sec)
    print('  tm_wday :', s.tm_wday)
    print('  tm_yday :', s.tm_yday)
    print('  tm_isdst:', s.tm_isdst)


now = time.ctime(1483391847.433716)
print('Now:', now)

parsed = time.strptime(now)
print('\nParsed:')
show_struct(parsed)

print('\nFormatted:',
      time.strftime("%a %b %d %H:%M:%S %Y", parsed))

# The output string is not exactly like the input, since the day of the month is prefixed with a zero.

Now: Mon Jan  2 23:17:27 2017

Parsed:
  tm_year : 2017
  tm_mon  : 1
  tm_mday : 2
  tm_hour : 23
  tm_min  : 17
  tm_sec  : 27
  tm_wday : 0
  tm_yday : 2
  tm_isdst: -1

Formatted: Mon Jan 02 23:17:27 2017
